In [1]:
import pandas as pd 

df = pd.read_csv('industry_cat_filings.csv')
df.head(4)

,Unnamed: 0,year,file,industry,sic_code,sic_cat
0,0,1995,1995-02-10_60026,GUIDED MISSILES & SPACE VEHICLES & PARTS,3760,Manufacturing
1,1,1995,1995-02-10_897599,ELECTRONIC COMPONENTS & ACCESSORIES,3670,Manufacturing
2,2,1995,1995-03-13_63506,TRUCKING & COURIER SERVICES (NO AIR),4210,Transportation
3,3,1995,1995-03-16_106015,"OIL, GAS FIELD SERVICES, NBC",1389,Mining


In [2]:
df.sic_cat.unique()

array(['Manufacturing', 'Transportation', 'Mining',
       'Finance, Insurance and Real Estate', 'Wholesale Trade',
       'Construction', 'Services', 'Public Administration',
       'Retail Trade', 'Agriculture'], dtype=object)

In [5]:
# #find the paths for all files 

# import re
# import os
# import pandas as pd
# import unicodedata
# from bs4 import BeautifulSoup
# from datetime import datetime

# def find_relative_filenames(path,start_date,end_date):
#     start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
#     end_datetime = datetime.strptime(end_date, '%Y-%m-%d')
#     file_names_html = [path+'/'+fn for fn in os.listdir(path) if fn.lower().endswith('html') and datetime.strptime(fn[:10], '%Y-%m-%d')>=start_datetime and datetime.strptime(fn[:10], '%Y-%m-%d')<=end_datetime]
#     file_names_txt = [path+'/'+fn for fn in os.listdir(path) if fn.lower().endswith('txt') and datetime.strptime(fn[:10], '%Y-%m-%d')>=start_datetime and datetime.strptime(fn[:10], '%Y-%m-%d')<=end_datetime]
#     return file_names_html, file_names_txt

# filename_txt, filename_html = find_relative_filenames("data","1995-01-10","2022-12-10")
# all_paths = filename_txt + filename_html

# df1 = pd.DataFrame({'file_paths': all_paths})
# df2  = df
# df2['path'] = df2['file']

# import difflib 
# df2['path'] = df2['path'].apply(lambda x: difflib.get_close_matches(x, df1['file_paths'])[0])

# # df2.to_csv('industry_paths.csv')

KeyboardInterrupt: 

In [6]:
#df = df2
df = pd.read_csv("industry_paths.csv")

In [7]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'year', 'file', 'industry', 'sic_code',
       'sic_cat', 'path'],
      dtype='object')

In [18]:
#defining the keyworsd for comparables and dcf valuation methods 

# Different kind of comparables based valuation
comps = ['relative valuation', 'trading multiple', 'trading multiples', 'market multiple', 'market multiples', 'equity comparables', 'equity comps', 'comparable companies','comparable company','peer company','peer companies','peer group'] 

ev_rev = ['ev/revenue', 'ev / revenue', 'enterprise value/revenue', 'enterprise value / eevenue', 'ev to revenue', 'enterprise value to revenue']

ev_gp = ['ev/gross grofit', 'ev / gross profit', 'enterprise value/gross profit', 'enterprise value / gross profit', 'ev to gross profit', 'enterprise value to gross profit']

ev_ebitda = ['ev/ebitda', 'ev / ebitda', 'enterprise value/ebitda', 'enterprise value / ebitda', 'ev to ebitda', 'enterprise value to ebitda', 'enterprise value to earning before interest']

p_e = ['p/e', 'p / e', 'price to earnings']

p_nav = ['p/nav', 'p / nav', 'price to net asset value']

p_b = ['p/b', 'p / b', 'price to book']

#comps_ratios = ['EV/Revenue', 'EV/Gross Profit', 'EV/EBITDA', 'P/E', 'P/NAV', 'P/B', 'ratio']
comps_ratios = ev_rev + ev_gp + ev_ebitda + p_e + p_nav + p_b
comps_ratios

comps_complete = comps + comps_ratios

dcf = ['cost of capital','cost of equity','wacc','weighted average cost of capital','capm','capital asset pricing model', 'discounted cash flow analysis', 'discounted cash flow method', "present value of", "projection of", "discounted earnings", "net present value", "discounted earnings", "discounted cash flow"]



In [19]:
sic_category = ["Agriculture", "Mining", "Construction", "Manufacturing", 
                "Transportation", "Wholesale Trade", "Retail Trade", 
                "Finance, Insurance and Real Estate", "Services", "Public Administration"]


In [20]:
for cat in sic_category: 
    #subset for the industry 
    finance = df[df.sic_cat == cat]
    finance = finance.reset_index( drop = True)

    comp_score = []
    comp_deet_score = []
    dcf_score = []

    #for each file in the industry df 

    for k in range(finance.shape[0]):
        file_name = finance.path[k]
        my_file = open(file_name, "r")
        data = my_file.read() #reading the data
        data = data.lower()

        #count
        comp_count = 0 
        comps_deets = 0 
        dcf_count = 0 

        for c in comps:
            if c in data: 
                comp_count += 1 
            
        for cd in comps_ratios: 
            if cd in data:
                comps_deets += 1

        for d in dcf:
            if d in data: 
                dcf_count += 1


        comp_score.append(comp_count)
        comp_deet_score.append(comps_deets)
        dcf_score.append(dcf_count)

    finance['comp_general'] = comp_score
    finance['comp_detailed'] = comps_deets
    finance['dcf'] = dcf_score 

    finance.to_csv("fin_results/"+cat +'_keywords_results.csv')
